<a href="https://colab.research.google.com/github/CurioLytics/Emotional-Analysis/blob/main/data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Cài đặt thư viện dùng để xử lý, phát hiện hoặc loại bỏ emoji trong văn bản (vì nhiều câu trong dữ liệu có chứa emoji)
!pip install emoji

In [ ]:
#nhập các thư viện cần thiết sẽ được sử dụng để xử lý và phân tích văn bản
import pandas as pd
import numpy as np
import re
import emoji
import nltk
from collections import Counter
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet

pandas, numpy: Đọc dữ liệu CSV, xử lý bảng dữ liệu

re: Dùng biểu thức chính quy để làm sạch văn bản

emoji: Phát hiện và xử lý emoji

nltk: Xử lý ngôn ngữ tự nhiên

PorterStremmer: Rút gọn từ về gốc (stemming)

WordNetLemmatizer: Lemmatization – đưa từ về dạng từ điển

Counter: Thống kê tần suất từ



In [ ]:
# Import necessary libraries
#requests: Thư viện này được sử dụng để tải xuống các tệp tin từ internet.
#os: Thư viện này cung cấp các hàm để tương tác với hệ điều hành.
#io: Đối tượng StringIO từ thư viện này được sử dụng để coi nội dung tập dữ liệu đã tải xuống như một tệp tin, cho phép pandas đọc nó một cách chính xác.
import requests
import os
from io import StringIO
#Tải và hiển thị dữ liệu từ GoEmotions
url = "https://storage.googleapis.com/gresearch/goemotions/data/full_dataset/goemotions_1.csv"

# sử dụng thư viện requests để gửi yêu cầu đến URL được chỉ định, yêu cầu dữ liệu từ vị trí đó. Kết quả của yêu cầu này được lưu trữ trong biến response
response = requests.get(url)
if response.status_code == 200:
    # Create a StringIO object from the response content
    csv_data = StringIO(response.text)

    # Read the CSV data into a pandas DataFrame
    df_train = pd.read_csv(csv_data)

    # Display basic information about the dataset
    print(f"Dataset shape: {df_train.shape}")
    print("\nFirst 5 rows of the dataset:")
    print(df_train.head())



Dataset shape: (70000, 37)

First 5 rows of the dataset:
                                                text       id  \
0                                    That game hurt.  eew5j0j   
1   >sexuality shouldn‚Äôt be a grouping category...  eemcysk   
2     You do right, if you don't care then fuck 'em!  ed2mah1   
3                                 Man I love reddit.  eeibobj   
4  [NAME] was nowhere near them, he was by the Fa...  eda6yn6   

                author            subreddit    link_id   parent_id  \
0                Brdd9                  nrl  t3_ajis4z  t1_eew18eq   
1          TheGreen888     unpopularopinion  t3_ai4q37   t3_ai4q37   
2             Labalool          confessions  t3_abru74  t1_ed2m7g7   
3        MrsRobertshaw             facepalm  t3_ahulml   t3_ahulml   
4  American_Fascist713  starwarsspeculation  t3_ackt2f  t1_eda65q2   

    created_utc  rater_id  example_very_unclear  admiration  ...  love  \
0  1.548381e+09         1                 False          

Dữ liệu lấy từ Google Cloud
Lưu vào df_train.
Thông tin dữ liệu là văn bản được người dùng viết trên Reddit, gồm 37 cột, 28 cột cảm xúc, 7000 dòng


In [ ]:
# Tạo ra một biến mới có tên là train_df và gán cho nó giá trị của biến df_train đã tồn tại
train_df=df_train

In [ ]:
#Hiển thị tên các cột trong tập dữ liệu, gồm 37 cột trong đó 28 cột là cảm xúc
print(train_df.shape)
cols = train_df.columns
print(cols)

(70000, 37)
Index(['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')


In [ ]:
# tách biệt và đếm số lượng các hàng trong DataFrame train_df được đánh dấu là "rất không rõ ràng" (very unclear).
unclear = train_df[train_df['example_very_unclear']==True]
print(unclear.shape)

(1129, 37)


In [ ]:
#Phân nhóm các nhãn cảm xúc thành 3 nhóm chỉnh
pos_labels = ['admiration','approval', 'amusement', 'caring', 'desire',
              'excitement', 'gratitude', 'joy', 'love','optimism', 'pride', 'relief']
neg_labels = ['anger', 'annoyance', 'disappointment', 'disapproval', 'disgust',
              'embarrassment','fear', 'grief', 'nervousness', 'remorse', 'sadness']
ambi_labels = ['confusion', 'curiosity', 'realization', 'surprise']

In [ ]:
#Đếm số lượng cảm xúc và nhóm cảm xúc
emotion_count = {}
category_count = {}
#Với mỗi nhãn trong pos_labels, tìm số dòng có nhãn đó (== 1) và lưu vào emotion_count.
#Đồng thời, cộng dồn số dòng thuộc nhóm positive vào pos_sum.
#Làm tương tự cho nhóm negative và ami_labels
pos_sum = 0
for label in pos_labels:
    df = train_df[train_df[label]==1]
    emotion_count[label] = len(df)
    pos_sum += len(df)
category_count['positive'] = pos_sum

neg_sum = 0
for label in neg_labels:
    df = train_df[train_df[label]==1]
    emotion_count[label] = len(df)
    neg_sum += len(df)
category_count['negative'] = neg_sum

ambi_sum = 0
for label in ambi_labels:
    df = train_df[train_df[label]==1]
    emotion_count[label] = len(df)
    ambi_sum += len(df)

df = train_df[train_df['neutral']==1]
emotion_count['neutral'] = len(df)
ambi_sum += len(df)
category_count['neutral'] = ambi_sum

In [ ]:
#Đếm số lượng cảm xúc
emotion_count


{'admiration': 5647,
 'approval': 5928,
 'amusement': 3081,
 'caring': 1988,
 'desire': 1248,
 'excitement': 1900,
 'gratitude': 3863,
 'joy': 2607,
 'love': 2745,
 'optimism': 2887,
 'pride': 452,
 'relief': 452,
 'anger': 2589,
 'annoyance': 4443,
 'disappointment': 2771,
 'disapproval': 3774,
 'disgust': 1704,
 'embarrassment': 817,
 'fear': 1048,
 'grief': 227,
 'nervousness': 598,
 'remorse': 849,
 'sadness': 2193,
 'confusion': 2471,
 'curiosity': 3267,
 'realization': 2867,
 'surprise': 1806,
 'neutral': 18423}

In [ ]:
#Đếm số lượng cảm xúc theo 3 nhóm
category_count

{'positive': 32798, 'negative': 21013, 'neutral': 28834}

In [ ]:
#Nhóm cảm xúc liên quan vào 5 nhóm chính
anger_list = [ "anger", "annoyance", "disapproval", "disgust"]
fear_list = ["fear", "nervousness"]
joy_list = ["joy", "amusement", "approval", "excitement", "gratitude","love", "optimism", "relief", "pride", "admiration", "desire", "caring"]
sadness_list = ["sadness", "disappointment", "embarrassment", "grief", "remorse"]
surprise_list = ["surprise", "realization", "confusion", "curiosity"]

In [ ]:
emotion_groups = [anger_list, fear_list, joy_list, sadness_list, surprise_list]

In [ ]:
# Nhóm các cảm xúc và tạo DataFrame mới
#Tạo mapping cảm xúc vào 5 nhóm chính: Anger, Fear, Joy, Sadness, Surprise, Neutral
"""
Labels:
Anger (0) : [“Anger”, “annoyance”, “disapproval”, “disgust”]
Fear (1) : [“fear”, “nervousness” ]
Joy (2) : [“joy” , “amusement”, “approval”, “excitement”, “gratitude”,
     “love”, “optimism”, “relief”, “pride”, “admiration”, “desire”, “caring”]
Sadness (3) : [“Sadness”, “Disappointment”, “Embarrassment”, “grief”, “remorse”]
Surprise (4) : [“Surprise”, “Realization”, “confusion”, “curiosity”]
Neutral (5) : ["Neutral"]
"""

col_names = ['text','group_label']
#Danh sách trống này sẽ lưu trữ dữ liệu cho DataFrame mới.
new_data = []
#Vòng lặp này lặp qua từng hàng (row) của DataFrame gốc (train_df), cùng với chỉ số của nó (id).
for id,row in train_df.iterrows():
  #Lọc các dòng không rõ ràng, bỏ qua bằng continue
    if row['example_very_unclear'] == True:
        continue
    else:
      #Nếu cột neutral là True, hàng sẽ được gán vào nhóm "Trung lập" (nhãn 5).
      #Nếu không, mã sẽ xác định nhóm cảm xúc chiếm ưu thế cho hàng bằng cách:
      #Lặp qua từng nhóm cảm xúc (eg) và đếm số lượng các cảm xúc riêng lẻ trong nhóm đó có trong hàng.
      #Gán nhóm có số lượng cao nhất (max_cnt) làm group_label của hàng (max_label).
        if row['neutral'] == True:
            data = [row['text'], 5]
        else:
            max_cnt = -1
            max_label = -1
            for ix,eg in enumerate(emotion_groups):
                cnt = 0
                for label in eg:
                    if row[label] == 1:
                        cnt += 1
                if cnt > max_cnt:
                    max_cnt = cnt
                    max_label = ix
            data = [row['text'], max_label]
        new_data.append(data)

#Tạo DataFrame mới:
emotion_group_train = pd.DataFrame(np.array(new_data),columns=col_names)
emotion_group_train.head()

,text,group_label
0,That game hurt.,3
1,"You do right, if you don't care then fuck 'em!",5
2,Man I love reddit.,2
3,"[NAME] was nowhere near them, he was by the Fa...",5
4,Right? Considering it‚Äôs such an important do...,2


In [16]:
emotion_group_train.shape

(68871, 2)

## NLP preprocessing

In [17]:
import pandas as pd
import re
import string
import nltk

# Tải stopwords tiếng Anh (nếu chưa có)
nltk.download("stopwords")
from nltk.corpus import stopwords

# Danh sách các từ dừng (có thể mở rộng nếu cần)
stop_words = set(stopwords.words("english"))

# 🛠 Hàm xóa URL
def remove_urls(text):
    return re.sub(r"http.?://[^\s]+[\s]?", "", text)

# 🛠 Hàm xóa @mentions
def remove_mentions(text):
    return re.sub(r"@\w+", "", text)

# 🛠 Hàm xóa dấu câu
def remove_punctuation(text):
    return text.translate(str.maketrans("", "", string.punctuation))

# 🛠 Hàm chuyển thành chữ thường
def to_lower(text):
    return text.lower()

# 🛠 Hàm xóa khoảng trắng thừa
def remove_whitespaces(text):
    return text.strip()

# 🛠 Hàm xóa số
def remove_digits(text):
    return re.sub(r"\d+", "", text)

# 🛠 Hàm xóa từ dừng
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return " ".join(filtered_words)

# 🛠 Hàm loại bỏ ký tự lặp (ví dụ: "coooool" -> "cool")
def de_repeat(text):
    pattern = re.compile(r"(.)\1{2,}")  # Bắt ký tự lặp từ 3 lần trở lên
    return pattern.sub(r"\1\1", text)  # Thay thế bằng chính nó nhưng chỉ lặp 2 lần

# 🛠 Hàm tổng hợp để làm sạch dữ liệu
def clean_text(text):
    if pd.isnull(text):  # Kiểm tra nếu giá trị NaN
        return ""
    text = to_lower(text)
    text = remove_mentions(text)
    text = remove_urls(text)
    text = remove_punctuation(text)
    text = remove_digits(text)
    text = de_repeat(text)
    text = remove_whitespaces(text)
    text = remove_stopwords(text)
    return text

# 🛠 Áp dụng vào DataFrame `emotion_group_train`
# Giả sử df đã được load sẵn
# df = pd.read_csv("emotion_group_train.csv")  # Nếu cần đọc từ file CSV

# Áp dụng làm sạch dữ liệu
emotion_group_train["clean_text"] = emotion_group_train["text"].apply(clean_text)

# Hiển thị kết quả
print(emotion_group_train.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


                                                text group_label  \
0                                    That game hurt.           3   
1     You do right, if you don't care then fuck 'em!           5   
2                                 Man I love reddit.           2   
3  [NAME] was nowhere near them, he was by the Fa...           5   
4  Right? Considering it‚Äôs such an important do...           2   

                                          clean_text  
0                                          game hurt  
1                            right dont care fuck em  
2                                    man love reddit  
3                           name nowhere near falcon  
4  right considering it‚äôs important document kn...  


## Deep Cleaning

In [18]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Danh sách emoticons
EMOTICONS = {
    ":)": "smile", ":-)": "smile",
    ":(": "sad", ":-(": "sad",
    ":D": "laugh", ":-D": "laugh",
    ";)": "wink", ";-)": "wink",
    ":P": "playful", ":-P": "playful"
}

# Tạo regex pattern để tìm emoticon
emoticon_pattern = r"|".join(re.escape(e) for e in EMOTICONS.keys())

### 1️.Kiểm tra nội dung đặc biệt ###
def contains_hashtag(text):
    """Kiểm tra xem văn bản có chứa hashtag không."""
    return bool(re.search(r"#\s", text)) if isinstance(text, str) else False

def contains_emoji(text):
    """Kiểm tra xem văn bản có chứa emoji không."""
    return bool(emoji.emoji_count(text)) if isinstance(text, str) else False

def contains_emoticon(text):
    """Kiểm tra xem văn bản có chứa emoticon không."""
    return bool(re.search(emoticon_pattern, text)) if isinstance(text, str) else False

### 2️. Xử lý emoji & emoticon ###
def convert_emoji_to_text(text):
    """Chuyển emoji thành từ mô tả (vd: 😍 -> [heart_eyes])"""
    if not isinstance(text, str):
        return text
    return emoji.demojize(text, delimiters=("[", "]"))

def convert_emoticon_to_text(text):
    """Chuyển emoticon thành mô tả (vd: :) -> smile)"""
    for emot in EMOTICONS:
        text = re.sub(re.escape(emot), "_".join(EMOTICONS[emot].split()), text)
    return text


### 3. Stemming & Lemmatization ###
def get_wordnet_pos(word):
    """Chuyển đổi POS tag thành dạng phù hợp với WordNet"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def stemming(text):
    """Chuyển từ về dạng gốc (running -> run, better -> good)"""
    ps = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemma_words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
    stemmed_words = [ps.stem(word) for word in lemma_words]
    return " ".join(lemma_words)

### 4. Hàm tổng hợp: Deep Cleaning ###
def deep_clean_text(text):
    """Áp dụng tất cả các bước deep cleaning vào văn bản"""
    if pd.isnull(text):
        return ""

    text = convert_emoji_to_text(text)  # Chuyển emoji thành text
    text = convert_emoticon_to_text(text)  # Chuyển emoticon thành text
    text = stemming(text)  # Stemming & Lemmatization

    return text

### 5. Áp dụng vào DataFrame ###
# Giả sử `emotion_group_train` là DataFrame chứa cột 'text'
# df = pd.read_csv("emotion_group_train.csv")  # Nếu cần đọc từ file CSV

# Tạo cột kiểm tra hashtag, emoji, emoticon
emotion_group_train["has_hashtag"] = emotion_group_train["clean_text"].apply(contains_hashtag)
emotion_group_train["has_emoji"] = emotion_group_train["clean_text"].apply(contains_emoji)
emotion_group_train["has_emoticon"] = emotion_group_train["clean_text"].apply(contains_emoticon)

# Áp dụng Deep Cleaning
emotion_group_train["deep_clean_text"] = emotion_group_train["clean_text"].apply(deep_clean_text)

# Hiển thị kết quả
print(emotion_group_train.head())


                                                text group_label  \
0                                    That game hurt.           3   
1     You do right, if you don't care then fuck 'em!           5   
2                                 Man I love reddit.           2   
3  [NAME] was nowhere near them, he was by the Fa...           5   
4  Right? Considering it‚Äôs such an important do...           2   

                                          clean_text  has_hashtag  has_emoji  \
0                                          game hurt        False      False   
1                            right dont care fuck em        False      False   
2                                    man love reddit        False      False   
3                           name nowhere near falcon        False      False   
4  right considering it‚äôs important document kn...        False      False   

   has_emoticon                                    deep_clean_text  
0         False                          

In [ ]:

# Chọn các cột mong muốn và đổi tên
cleaned_data = emotion_group_train[['deep_clean_text', 'group_label']].rename(columns={'deep_clean_text': 'text', 'group_label': 'emotion'})

# Xuất dữ liệu ra file CSV
cleaned_data.to_csv('cleaned_data.csv', index=False)
